In [1]:
import ast
import os

import pandas as pd
from Bio import Entrez

from scripts.data_processing import analyze_exons
from scripts.fasta_processing import read_single_fasta, dict_align_to_fasta
from scripts.parse_psi_blast_results import create_many_cassettes, dict_align_info_analyze

Entrez.email = "artemvaskaa@gmail.com"

## Переопределение функций

In [2]:
def find_cassettes(df):
    ref_exon_len = [37]
    found_protein_ids = {}

    for protein_id in df.protein_id:
        df_exons = analyze_exons(f"{dir}/{protein_id}/exons.fa")
        condition = set(ref_exon_len) & set(df_exons.length.tolist())
        if condition:
            found_protein_ids[protein_id] = condition
    df_found_protein_ids = df[df["protein_id"].isin(found_protein_ids.keys())]  # !!!

    data = {}
    for protein_id in df_found_protein_ids.protein_id:
        df_subset = df_found_protein_ids[df_found_protein_ids["protein_id"] == protein_id]
        org_name = df_subset.org_name.iloc[0]
        df_exons = analyze_exons(f"{dir}/{protein_id}/exons.fa")
        exon_37_idx = df_exons[df_exons["length"] == 37].index[0]
        exon_110_idx = exon_37_idx - 1
        data[f"{org_name}__{protein_id}"] = (df_exons, [exon_110_idx, exon_37_idx])

    return data, df_found_protein_ids

## Введение

In [3]:
# читаем все созданные таблицы
datasets_Actinopterygii = pd.read_csv("../results_summary/datasets_Actinopterygii_cds_cassette.tsv", sep="\t")
datasets_Amphibia = pd.read_csv("../results_summary/datasets_Amphibia_cds_cassette.tsv", sep="\t")
datasets_Sauropsida = pd.read_csv("../results_summary/datasets_Sauropsida_cds_cassette.tsv", sep="\t")

# объединяем их в одну
datasets_Deuterostomia = pd.concat([datasets_Actinopterygii, datasets_Amphibia, datasets_Sauropsida], ignore_index=True)

# смотрим на value_counts в полученных данных из datasets, чтобы понять, что бластить
datasets_Deuterostomia.sub_phylum.value_counts()

sub_phylum
Euteleosteomorpha    53
Aves                 36
Lepidosauria         28
Otomorpha            16
Testudinata          15
Anura                14
Gymnophiona           3
Osteoglossomorpha     2
Caudata               2
Crocodylia            2
Cladistia             1
Name: count, dtype: int64

## Gymnophiona

In [4]:
datasets_Deuterostomia[datasets_Deuterostomia["sub_phylum"] == "Gymnophiona"]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
72,Gymnophiona,geotrypetes_seraphini_2,Geotrypetes_seraphini,894,False,839,55,3065,110,datasets
73,Gymnophiona,microcaecilia_unicolor_0,Microcaecilia_unicolor,1116,False,929,187,2784,110,datasets
74,Gymnophiona,rhinatrema_bivittatum_1,Rhinatrema_bivittatum,1134,False,1031,103,4053,110,datasets


[geotrypetes_seraphini_gymnophiona](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_033810741.1%20NXF1%20%5Borganism%3DGeotrypetes%20seraphini%5D%20%5BGeneID%3D117365020%5D%0AMSETLETPKGRYTPYDRRRRRGEDWHDRDRGGTSNMQFIVRRDAGHPDRPGGNREPRKTWHKITIPHGKK%0AYEKNWLLSAIQSVCTVPFTPVEFHHDHNRALFYVEEASTAAALRQASRRITDKDGYKVTILSNPSDIPQT%0AILKELKPEEIVMLKVCMSKRYDSELQALDLKNLRNDPELLCNNIDLVLNRKNCLLTVLRIIEEHIPELLS%0ALNLSSNKLYRLDELTELVRKAPNLKILNLFCNELKSDRELDKVKGLKLEELWLDGNPLCDAFKDQSTYIR%0AAMRERFPMLLRLDGHILPPPIAFDVEAPTTLPPCKGSYFASDDIKALVLRFLQQYYSVYDSEDRQGLLDA%0AYHDGACCSLCIPAALGNPNRNNLSEYFKDSRNVKRLKDPAMRLKLLKHTRLNVVAFLNELPKTLHDTNSF%0AVVDICAQTNMLLSFVVHGIFKEVDGKSRDEVRAFSRMFVAVPAGNAGLCIVNDELFVRNATTEEIRKAFA%0ATPAPTPSSSPVPTLTATHQELLQAFSLQSGMNLEWSQKCLQDNDWDFPRAAQIFTQLKAEGKIPEVAFMK&JOB_TITLE=geotrypetes_seraphini_gymnophiona&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Gymnophiona%20%28taxid%3A8445%29&EQ_MENU1=Geotrypetes%20seraphini%20(taxid:260995)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

In [5]:
phylum = "Gymnophiona"
dir = f"../Sequences_protein_id/{phylum}"

In [6]:
# # читаем результат BLAST
# name_of_blast_res = f"../blast_res/psi_blast/geotrypetes_seraphini_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [7]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [8]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [9]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

array([ True])

In [10]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds_cassette", dir)
df_cds_cassette

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Gymnophiona,Rhinatrema_bivittatum,Rhinatrema_bivittatum__XP_029448917.1,918,False,881,37,6770,110,psi_blast
1,Gymnophiona,Rhinatrema_bivittatum,Rhinatrema_bivittatum__XP_029448920.1,918,False,881,37,6770,110,psi_blast
2,Gymnophiona,Rhinatrema_bivittatum,Rhinatrema_bivittatum__XP_029448918.1,918,False,881,37,6770,110,psi_blast


In [11]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Gymnophiona,Rhinatrema_bivittatum,Rhinatrema_bivittatum__XP_029448917.1,918,False,881,37,6770,110,psi_blast
1,Gymnophiona,Rhinatrema_bivittatum,Rhinatrema_bivittatum__XP_029448920.1,918,False,881,37,6770,110,psi_blast
2,Gymnophiona,Rhinatrema_bivittatum,Rhinatrema_bivittatum__XP_029448918.1,918,False,881,37,6770,110,psi_blast


In [12]:
# удаляем дубликаты

# -----------------
# 0, 2 - Rhinatrema_bivittatum__XP_029448917.1, Rhinatrema_bivittatum__XP_029448918.1
# -----------------

indices_to_drop = [0, 2]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [13]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Geotrypetes_seraphini_reference"] = read_single_fasta("../datasets/Gymnophiona/geotrypetes_seraphini_2/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [14]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Osteoglossomorpha

In [15]:
datasets_Deuterostomia[datasets_Deuterostomia["sub_phylum"] == "Osteoglossomorpha"]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
69,Osteoglossomorpha,paramormyrops_kingsleyae_0,Paramormyrops_kingsleyae,1254,False,1133,121,2929,110,datasets
70,Osteoglossomorpha,scleropages_formosus_1,Scleropages_formosus,1239,False,1127,112,3412,110,datasets


[paramormyrops_kingsleyae_osteoglossomorpha](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_023650121.1%20nxf1a%20%5Borganism%3DParamormyrops%20kingsleyae%5D%20%5BGeneID%3D111834735%5D%20%5Bisoform%3DX2%5D%0AMTFFRKRYDNLAVVAYIHPDEDCDGVSFIQREVRSNGRLLEEHDDRVGGPQFRNRKGRGPFRGRMYSEQV%0AGRPRNRVGPGGGGPNPRTRFEDDDGDVAMNDSSQDGTPQRRYNPYDRPNRRGHNRPDRDRRGGGGGGGGY%0AGGGPAGGGGGGGAGRKNWFRMTIPHGKKYEKKWLLTVLKDNCAMPFTPIHYQTEGNKVQFYIEDATTANA%0ALQKVSRKITDTEGYKVTVLMNSCPPPAFIQTELKPEDLEHLKQCMAKRFDGSQQALDLNSIRTDPDLVSQ%0ANIDVVLNRKNCMQAVIKIIEENIPELVCLNLSNNRLYKLDDLAELISKAPNLKTLNLSQNELRTERELDK%0AIKGFKLVEIWLDRNPLCDHFKDQPNYISAVRERFPRVLKLDGHDLPPPIGFDVEAPTTLPPCKGSYFVSD%0AEIKSLILRFLQQYYNCYDSADRQPLLEAYHDGACFSLSIPFSLQNPSRSSLGDYHKDSRNIKKLKDPTTR%0AFRLLKHTRLNVVAFLNELPKTQHDTASFTVDVSTYTNTLLSFTVSGVFKEVDGRSRDAVRAFSRIFIAVP%0AAGNAGLCIVNDELFVRNATTEEIRKAFAAPAPTPSSSPVPTLSAPQQEMLSSFSQKSGMNLEWSQKCLQD%0ANDWDFNRAAEIFTQLKAQGKIPDVAFIK&JOB_TITLE=XP_023650121.1%20nxf1a%20%5Borganism%3DParamormyrops...&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Osteoglossomorpha%20%28taxid%3A31089%29&EQ_MENU1=Paramormyrops%20kingsleyae%20(taxid:1676925)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [16]:
phylum = "Osteoglossomorpha"
dir = f"../Sequences_protein_id/{phylum}"

In [17]:
# # читаем результат BLAST
# name_of_blast_res = f"../blast_res/psi_blast/paramormyrops_kingsleyae_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [18]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [19]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [20]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
# df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
# df_cds.equal_to_cds.unique()

# падает с ошибкой, потому что нашелся всего 1 вид и старт-кодон не в начале последовательности
# дальше проводить анализ нет смысла

## Caudata

In [21]:
datasets_Deuterostomia[datasets_Deuterostomia["sub_phylum"] == "Caudata"]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
75,Caudata,ambystoma_mexicanum_0,Ambystoma_mexicanum,1068,False,1067,1,10340,110,datasets
76,Caudata,pleurodeles_waltl_1,Pleurodeles_waltl,1344,False,1193,151,3245,110,datasets


[ambystoma_mexicanum_caudata](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_069501258.1%20NXF1%20%5Borganism%3DAmbystoma%20mexicanum%5D%20%5BGeneID%3D138567183%5D%0AMGRRAGPTAPPIDGRSFEHDDRVGGGFPMKKRKGRGPFRGKMYSGEQGHRGRNRGAGGAPPRPRLEDRDG%0ADVAMNDTLDNPRNRYNPYGHRPSRHEDWSARDQGGPSNIHITVRRDMPPQDQRGGSRENRKQWFRITIPQ%0AGRKYEKTWILSAIQSACTVPFIPIEFHFDRMRAEFYVEGENVAKALQQLSRRITDKDDNKMVILIKSCNP%0APPSTQKEFQDLKDEDIVQLKLCMSRRYDGALQALDLKNVRSDTELMTQGVVVMLQKRSCMEAVLRIIEEN%0AIPELLALDLSNNKLCRLEDLTELPQKAPLLKHLNLSGNELKTERELDRLKGMKLQELWLDGNPLCDFFRD%0AQSSYISAIRERFPKLLRLDGHDLPPPIAFDVETPTTLPPSKGSYFGSDDIKLLVLRFLHQYYSVYDSGNR%0AQALLDAYHDGACCSINIPFFTQNQNRNNLSDYFKDSRNVKKQKDPIMRFKLLKHTRLSVVAFLNELPKTL%0AHDINSFVIDITAQTNTLLCFAVHGVFKEVEGKFRDSTRAFNRVFIAVPAANAGLCIVNDQLHVRQATTEE%0AIRKAFVTPAPTPSSSPVLTLPSTHQEMLQAFSLQSGMNLEWSQKCLIDNDWDFARSAQIFTQLKLEGKIP%0ADVAFLK&JOB_TITLE=ambystoma_mexicanum_caudata&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Caudata%20%28taxid%3A8293%29&EQ_MENU1=Ambystoma%20mexicanum%20(taxid:8296)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [22]:
phylum = "Caudata"
dir = f"../Sequences_protein_id/{phylum}"

In [23]:
# # читаем результат BLAST
# name_of_blast_res = f"../blast_res/psi_blast/ambystoma_mexicanum_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0.8, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [24]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [25]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [26]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, phylum, "cds", dir)
df_cds.equal_to_cds.unique()

array([ True])

In [27]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, phylum, "cds_cassette", dir)
df_cds_cassette

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Caudata,Pleurodeles_waltl,Pleurodeles_waltl__KAJ1108515.1,1245,False,1094,151,3245,110,psi_blast


In [28]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source


In [29]:
# удаляем дубликаты

# -----------------

# -----------------

# indices_to_drop = []
# df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [30]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Geotrypetes_seraphini_reference"] = read_single_fasta("../datasets/Gymnophiona/geotrypetes_seraphini_2/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
# dict_align_to_fasta(dict_align_cds_cassette, f"../alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [31]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Crocodylia

In [32]:
datasets_Deuterostomia[datasets_Deuterostomia["sub_phylum"] == "Crocodylia"]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
134,Crocodylia,alligator_sinensis_1,Alligator_sinensis,999,False,851,148,1497,110,datasets
135,Crocodylia,alligator_mississippiensis_0,Alligator_mississippiensis,1293,False,1145,148,1618,110,datasets


[alligator_sinensis_crocodylia](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_025070745.1%20NXF1%20%5Borganism%3DAlligator%20sinensis%5D%20%5BGeneID%3D102374093%5D%0AMSDVHDGPRCRYNPYGSRPNRRGDDWHDRGRGGASNIHVAARRDQPIPDRGGNAGRDGTRKNWFKITIPY%0AGRKYDRSWLLTAIQNVCTVPFNPVEFHYDHNRAQFYVEDATAASALKLVSRKITDRENYKVVIIINSSAP%0APQSVQNELKPEEIEHLKLCMSKRYDRLQQALDLKDLRTDSDLVAQNVDMVLKRCSCMQAVLRIIEENIIP%0AELLSLNLSGNKLYRLDDLAELAQKATNIKILNLSRNELKSERELDKVKGLKLEELWLDGNPLCDCFRDQS%0ATYISAVRERFPKLLRLDGHELPPPIAFDVEAPTTLPPCKGSYFGSDNVKVLALRFLQLYYSIYDSGDRQG%0ALLDAYHDGACCSLSIPFFPQNPSRSTLSEYFKDSRNVKKLKDPTMRFKLLKHTRLNVVAFLNELPKTQHD%0AINSFVVDVCAQTNTLLCFAVHGIFKEVDGKSRDSVRAFTRVFVVVPAGNSGLCIVNDQLFVRNATTEEIR%0AKAFVMPAPTPSSSPVPTLSAEQQEMLQAFSMQSGMNLEWSQKCLQDNDWDYSRAAQVFTQLKVTQGPPVG%0AVGWGGRRVRGTPRGGWEGTPRVTSALSQ&JOB_TITLE=alligator_sinensis_crocodylia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Crocodylia%20%28taxid%3A8493%29&EQ_MENU1=Alligator%20sinensis%20(taxid:38654)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

No significant similarity found.

## Cladistia

In [33]:
datasets_Deuterostomia[datasets_Deuterostomia["sub_phylum"] == "Cladistia"]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
71,Cladistia,erpetoichthys_calabaricus_0,Erpetoichthys_calabaricus,1173,False,1118,55,3662,110,datasets


[erpetoichthys_calabaricus_cladistia](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_028665628.1%20nxf1b%20%5Borganism%3DErpetoichthys%20calabaricus%5D%20%5BGeneID%3D114657852%5D%20%5Bisoform%3DX1%5D%0AMAFRRRRFQNFQVVRQVHFEEHGVYDGRQEAPFLPIGYSEHDDRVGGPTFRTKKGRGPFRGKMYSEQTSR%0AQRYRGGGAPGPRSRYEDDDDGDVSMSESLHDSGQRRYMPYGTRPGRRGEERHDNDRDRAGTSNNLSGLRL%0ASPREGSSRKGWFKVTIPFGKKYSNSWLLSTLQGLCMAPFEPVQLHYEGQKAVFYVEDSNIANALKASSKK%0AIRDKENFKLIILVRSSSPPSSVVRQELLKPEAVEILKQCMSKRYDRTQQALDLSNLRSDPDLLSQNIDAV%0ALNRKNIMQAVVKIIEEYIPELLSLNISNNRLYKLSNLTDLVGKAPNLKILNLSQNELKSDRELDRIKGFK%0ALEELWLDGNPLCDFSKDQTAYIRVVRERFPKLLRLDGHDLPLPITFDVEAPTTLPPCKGSYFGSDEIKSL%0AVLRFLHQYFYVYDSGDRQALLDAYHDGASFSVCIPISLPNPSRNALGEYHKDSRNLKKVKDPTNRFRMLK%0AHTRLNVVAFLNELPKTQHDVNSFVVDVHAYTNMLLSFTVNGVFKEVDDKSPHSVRAFTRAFIAVPSGNSG%0ALCIVNDELFVRKASSEEIRKAFATPAPTPSSSPVPTLTARQQEMLTEFSQRSGMNLEWSQKCLLDNDWDY%0ATQAGQVFEKLNAEGKIPEVAFKQYKLQ&JOB_TITLE=erpetoichthys_calabaricus_cladistia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Cladistia%20%28taxid%3A8288%29&EQ_MENU1=Erpetoichthys%20calabaricus%20(taxid:27687)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [34]:
phylum = "Cladistia"
dir = f"../Sequences_protein_id/{phylum}"

In [35]:
# # читаем результат BLAST
# name_of_blast_res = f"../blast_res/psi_blast/erpetoichthys_calabaricus_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [36]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [37]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [38]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
# df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
# df_cds.equal_to_cds.unique()

# у единственного вида не было найдено стоп-кодона, поэтому дальнейший анализ не имеет смысла